In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [3]:
PATH = ["../dataset/train.tsv","../dataset/valid.tsv","../dataset/test.tsv", "../dataset/augm_helper.tsv"]

In [4]:
train = pd.read_csv(PATH[0], sep="\t")
valid = pd.read_csv(PATH[1], sep="\t")
test = pd.read_csv(PATH[2], sep="\t")
augm = pd.read_csv(PATH[3], sep="\t")

In [5]:
data = train.preprocessed_tweet_text_no_link
data = data.append(valid.preprocessed_tweet_text_no_link)
data = data.append(test.preprocessed_tweet_text_no_link)
data = data.append(augm.preprocessed_tweet_text_no_link)
data = data.reset_index(drop=True)
data.shape

(15668,)

In [6]:
vectorizer = TextVectorization(max_tokens=26000, output_sequence_length = 128, split='whitespace', standardize='lower_and_strip_punctuation', output_mode="int")
ds = tf.data.Dataset.from_tensor_slices(data)
vectorizer.adapt(ds.batch(64))

2022-01-13 12:11:23.534582: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
vectorizer.get_vocabulary()[:4]

['', '[UNK]', 'the', 'of']

In [8]:
train_data = train.preprocessed_tweet_text_no_link.to_list()
train_data += augm.preprocessed_tweet_text_no_link.to_list()
train_data = pd.DataFrame(train_data,columns=["preprocessed_tweet_text_no_link"])

In [10]:
train_vectors_text = vectorizer(train_data.preprocessed_tweet_text_no_link.to_numpy())

In [11]:
valid_vectors_text = vectorizer(train.preprocessed_tweet_text_no_link.to_numpy())

In [12]:
test_vectors_text = vectorizer(test.preprocessed_tweet_text_no_link.to_numpy())

In [13]:
np.save("vectors/X_train", train_vectors_text)
np.save("vectors/X_valid", valid_vectors_text)
np.save("vectors/X_test", test_vectors_text)

In [14]:
Y_train = train.q1_label.map(dict(yes=1, no=0))
Y_augm = augm.q1_label
Y_valid = valid.q1_label.map(dict(yes=1, no=0))
Y_test = test.q1_label.map(dict(yes=1, no=0))

In [15]:
np.save("vectors/Y_train_q1", np.array(Y_train.to_list() +  Y_augm.to_list()))
np.save("vectors/Y_valid_q1", Y_valid.to_numpy())
np.save("vectors/Y_test_q1", Y_test.to_numpy())

In [25]:
train.q2_label = train.q2_label.fillna("unk")
train.q3_label = train.q3_label.fillna("unk")
train.q4_label = train.q4_label.fillna("unk")